# Импорт библиотек

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tika

In [3]:
!pip install razdel

In [4]:
!pip install pymorphy2

In [5]:
!cp "/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/src/preprocessing_tools/preprocessing_tools.py" .

In [6]:
!cp "/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/src/preprocessing_tools/stopwords.txt" .

In [7]:
!cp "/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/src/parsers/xml_parser.py" .

In [8]:
import pandas as pd
import gensim
import re
import preprocessing_tools as pr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from xml_parser import XmlParser

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Получение данных из РПД

пример извлечения текста из документа


In [9]:
prep_tools = pr.preprocessing_tools()

In [10]:
file_data= pr.parser.from_file("/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/18_056866_Введение в блокчейн технологии_База.pdf")
text = file_data['content']
print(text)



















































  
  
  
  
  
  
  
  
  
  
  
  
  
  

Санкт-Петербургский государственный университет  
 
  
 
 

Р А Б О Ч А Я   П Р О Г Р А М М А  
УЧЕБНОЙ ДИСЦИПЛИНЫ  

 
 

Введение в блокчейн технологии  
An Introduction to Blockchain Technologies 

 
 

Язык(и) обучения 
  

русский 
 
 

Трудоемкость в зачетных единицах: 3 
  

Регистрационный номер рабочей программы: 056866 
  

  
  



Раздел 1. Характеристики учебных занятий 
 
1.1. Цели и задачи учебных занятий 
Основной целью освоения дисциплины «Введение в блокчейн технологии» является 
приобретение обучающимися знаний об основных понятиях, технологиях и способах 
применения технологии блокчейн, а также практических навыков работы с 
инструментами, применяемыми в области криптовалют. 
Поставленная цель достигается путем решения следующих задач курса: 
1) Ознакомить студентов с основными задачами, решаемыми в области криптовалют, 
базовыми алгоритмами этой области, а также со сферами 

Получение текстов из документов


In [11]:
RPD_texts = prep_tools.extract_text_from_pdfs_recursively('/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD')

Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001136_Технологии специализированных баз данных для моделирования.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001133_Базы знаний.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001112_Алгоритмические основы распознавания изображения_2018.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/001125_Устойчивость движений дискретных динамических систем.pdf
Processing /content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/RPD/5503_database_techologies_2019/018997_Оптимизация баз данных.pdf
Processing /content/drive/My Drive/Magist

Преобразование текстов в датафрейм


In [12]:
df = pd.DataFrame(RPD_texts, columns=['raw_text'], dtype='str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   raw_text  40 non-null     object
dtypes: object(1)
memory usage: 448.0+ bytes


In [13]:
df.head()

,raw_text
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


# Очистка текстов

In [14]:
df['clean_text'] = df['raw_text'].apply(prep_tools.clean_text)
df['clean_text_no_stopwords'] = df['clean_text'].apply(prep_tools.stop_words_remove)
df.head()

,raw_text,clean_text,clean_text_no_stopwords
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,р а б о ч а я п р о г р а м м а учебной дис...,"['учебный', 'дисциплина', 'технология', 'специ..."
1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,р а б о ч а я п р о г р а м м а учебной дис...,"['учебный', 'дисциплина', 'база', 'знание', 'k..."
2,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,р а б о ч а я п р о г р а м м а у ч е б н о...,"['алгоритмический', 'основа', 'распознавание',..."
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,р а б о ч а я п р о г р а м м а учебной дис...,"['учебный', 'дисциплина', 'устойчивость', 'дви..."
4,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,санктпетербургский государственный университет...,"['санктпетербургский', 'государственный', 'уни..."


In [15]:
tfidf = TfidfVectorizer(max_features=50000, ngram_range=(1, 2))
Vectorized_RPD = tfidf.fit_transform(df.clean_text_no_stopwords)
Vectorized_RPD

<40x29241 sparse matrix of type '<class 'numpy.float64'>'
	with 71840 stored elements in Compressed Sparse Row format>

# Матрица схожести

In [16]:
similarity_matrix = cosine_similarity(Vectorized_RPD)

получение данных из текстов профессиональных стандартов

In [17]:
professional_standarts_texts = XmlParser.extract_text_from_xmls('/content/drive/My Drive/Magistracy_SPbU/3_semester/Study_practice/professional_standarts/data/professional_standarts')

TypeError: ignored

In [ ]:
professional_standarts_texts

# Поиск наиболее релевантных документов

In [ ]:
# DATA


# MODEL PARAMETERS
dm = 1 # 1 for distributed memory(default); 0 for dbow 
cores = multiprocessing.cpu_count()
size = 300
context_window = 50
seed = 42
min_count = 1
alpha = 0.5
max_iter = 200

# BUILD MODEL
model = gensim.models.doc2vec.Doc2Vec(documents = documents,
dm = dm,
alpha = alpha, # initial learning rate
seed = seed,
min_count = min_count, # ignore words with freq less than min_count
max_vocab_size = None, # 
window = context_window, # the number of words before and after to be used as context
size = size, # is the dimensionality of the feature vector
sample = 1e-4, # ?
negative = 5, # ?
workers = cores, # number of cores
iter = max_iter) # number of iterations (epochs) over the corpus)